<a href="https://colab.research.google.com/github/lounisob/implementation_ml/blob/main/Projet_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SPEC CPU:

In [ ]:
!cat /proc/cpuinfo
print("----------")
!cat /proc/meminfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

Pour afficher les infos sur le GPU que Colab utilise (Tesla K80)

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Mar 25 06:35:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install ipython-autotime
%load_ext autotime

Version sequentielle de ViolaJones

Source:https://github.com/aparande/FaceDetection

In [ ]:
!python drive/MyDrive/FaceDetection/viola_jones.py 
%load_ext autotime

Computing integral images
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 7.18 s (started: 2022-03-25 06:39:50 +00:00)


Version GPU :

In [ ]:
!python drive/MyDrive/FaceDetection/viola_jones_gpu.py 
%load_ext autotime

Computing integral images
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 4.42 s (started: 2022-03-25 06:39:01 +00:00)


# Projet IMPLEMENTATION ML
Par Lounis Ould Bouali

Le code ci-dessous a été développer au cours des TPs:
La fonction transpose a été prises sur le github officiel de numba qui est une version optimisé.

In [1]:
from numba import cuda
from numba.cuda.cudadrv.driver import driver
from numba.np import numpy_support as nps
import numba as nb
import numpy as np
import math
from math import *


# Source: https://github.com/numba/numba/blob/main/numba/cuda/kernels/transpose.py
def transpose(a, b=None):
    """Compute the transpose of 'a' and store it into 'b', if given,
    and return it. If 'b' is not given, allocate a new array
    and return that.
    This implements the algorithm documented in
    http://devblogs.nvidia.com/parallelforall/efficient-matrix-transpose-cuda-cc/
    :param a: an `np.ndarray` or a `DeviceNDArrayBase` subclass. If already on
        the device its stream will be used to perform the transpose (and to copy
        `b` to the device if necessary).
    """

    # prefer `a`'s stream if
    stream = getattr(a, 'stream', 0)

    if not b:
        cols, rows = a.shape
        strides = a.dtype.itemsize * cols, a.dtype.itemsize
        b = cuda.cudadrv.devicearray.DeviceNDArray(
            (rows, cols),
            strides,
            dtype=a.dtype,
            stream=stream)

    dt = nps.from_dtype(a.dtype)

    tpb = driver.get_device().MAX_THREADS_PER_BLOCK
    # we need to factor available threads into x and y axis
    tile_width = int(math.pow(2, math.log(tpb, 2) / 2))
    tile_height = int(tpb / tile_width)

    tile_shape = (tile_height, tile_width + 1)

    @cuda.jit
    def kernel(input, output):

        tile = cuda.shared.array(shape=tile_shape, dtype=dt)

        tx = cuda.threadIdx.x
        ty = cuda.threadIdx.y
        bx = cuda.blockIdx.x * cuda.blockDim.x
        by = cuda.blockIdx.y * cuda.blockDim.y
        x = by + tx
        y = bx + ty

        if by + ty < input.shape[0] and bx + tx < input.shape[1]:
            tile[ty, tx] = input[by + ty, bx + tx]
        cuda.syncthreads()
        if y < output.shape[0] and x < output.shape[1]:
            output[y, x] = tile[tx, ty]

    # one block per tile, plus one for remainders
    blocks = int(b.shape[0] / tile_height + 1), int(b.shape[1] / tile_width + 1)
    # one thread per tile element
    threads = tile_height, tile_width
    kernel[blocks, threads, stream](a, b)

    return b

@cuda.jit
def kernel(s_tab,N,m,sum_tab):
      
      local_id = cuda.threadIdx.x
      blockIdx_x = cuda.blockIdx.x
      blockDimx_x = cuda.blockDim.x
      #global_id = cuda.grid(1)

      # Montée
      for d in range(0,m):
        cuda.syncthreads()
        k = local_id * 2**(d+1) + (blockIdx_x) * (blockDimx_x)
        if (local_id * 2**(d+1)<= N-1):
          s_tab[k+(2**(d+1)-1)]+= s_tab[k+(2**(d)-1)]
  
      cuda.syncthreads()
      if local_id == 0:
        sum_tab[blockIdx_x] = s_tab[((blockIdx_x) * (blockDimx_x))+blockDimx_x-1]
        s_tab[((blockIdx_x) * (blockDimx_x))+blockDimx_x-1]=0
     
      # Descente
      for d in range (m-1,-1,-1) :
          cuda.syncthreads()
          k = local_id * 2**(d+1) + (blockIdx_x) * (blockDimx_x)
          if (local_id * 2**(d+1) <= N-1):
            t = s_tab[k+(2**d)-1]
            s_tab[k+(2**d)-1] = s_tab[k+(2**(d+1))-1]
            s_tab[k+(2**(d+1))-1]+= t
  

def ScanCPU_Special(array,m,N):
    #m=int(math.log2(len(array))
    for d in range(0,m):
        for k in range (0,N-1,(2**(d+1))):
            array[k+(2**(d+1))-1]+= array[k+(2**d)-1]

    array[N-1]=0
    for d in range (m-1,-1,-1) :
        for k in range (0,N-1,2**(d+1)) :
            t = array[k+(2**d)-1]
            array[k+(2**d)-1]= array[k+(2**(d+1))-1]
            array[k+(2**(d+1))-1]+= t
    return array


def scanGPU(p):
  print("SCAN GPU",p)
  N=len(p) # taille du tableau
  print(N)
  m= int(log2(N)) # 2^m = N
  threadsPerBlock= 2 #1 seule  thread par grilles pour les tableau de petites tailles
  blocksPerGrid= math.ceil(N/threadsPerBlock)
  print("TPB",threadsPerBlock,"BPG",blocksPerGrid)
   
  if(log2(N)%2 != 0):  
    for i in range((N-threadsPerBlock),threadsPerBlock):
       p = np.append(p,0)
  print(p)
  s_tab = cuda.to_device(p)

  array2 = np.arange(0, blocksPerGrid, 1)
  sum_tab = cuda.to_device(array2)
  kernel[blocksPerGrid,threadsPerBlock](s_tab,threadsPerBlock,int(log2(threadsPerBlock)),sum_tab)
  cuda.synchronize()
  result = s_tab.copy_to_host()
  res_sum_tab = sum_tab.copy_to_host()
  res_copy = result.copy()
  scan_res_sum = ScanCPU_Special(res_sum_tab.copy(),int(log2(len(res_sum_tab))),len(res_sum_tab))

  print("RES_COPY",res_copy)    
  print("scan_res_sum",scan_res_sum)

  cpt = 0
  for i in range(0,len(res_copy)):
    if i>0 and i%threadsPerBlock==0:
      cpt = cpt+1
    temp = res_copy[i] + scan_res_sum[cpt]
    res_copy[i] = temp
  res_copy = res_copy[0:N]      
  return res_copy


def main():
  
    array = np.array([[5,3,2,4],[3,2,4,1],[4,8,3,1]])
    print("INITIAL",array)
    new_array = []
    #Pour chaque ligne de la matrice
    for p in array:
      new_array.append(scanGPU(p))
      
      # Si la taille du tableau est inférieur à 32
      """
      if (N<=32):
        result = ScanCPU_Special(array,int(log2(N)),N)
        print("( GPU : ",result, " )")
      else:
      """  
     
    np_arr = np.array(new_array)
    print("new array",np_arr)

    trspose_arr = transpose(np_arr)
    trspose_arr = trspose_arr.copy_to_host()
    step2_array = []
    print(trspose_arr)
    cuda.synchronize()
    
    for p in trspose_arr:
      step2_array.append(scanGPU(p))
    print(step2_array)
    step2_array = np.array(step2_array)
    new_trspose_arr = transpose(step2_array)
    new_trspose_arr = new_trspose_arr.copy_to_host()
    print(new_trspose_arr)

if __name__ == "__main__":
    main()


INITIAL [[5 3 2 4]
 [3 2 4 1]
 [4 8 3 1]]
SCAN GPU [5 3 2 4]
4
TPB 2 BPG 2
[5 3 2 4]
RES_COPY [0 5 0 2]
scan_res_sum [0 8]
SCAN GPU [3 2 4 1]
4
TPB 2 BPG 2
[3 2 4 1]
RES_COPY [0 3 0 4]
scan_res_sum [0 5]
SCAN GPU [4 8 3 1]
4
TPB 2 BPG 2
[4 8 3 1]
RES_COPY [0 4 0 3]
scan_res_sum [ 0 12]
new array [[ 0  5  8 10]
 [ 0  3  5  9]
 [ 0  4 12 15]]
[[ 0  0  0]
 [ 5  3  4]
 [ 8  5 12]
 [10  9 15]]
SCAN GPU [0 0 0]
3
TPB 2 BPG 2
[0 0 0 0]
RES_COPY [0 0 0 0]
scan_res_sum [0 0]
SCAN GPU [5 3 4]
3
TPB 2 BPG 2
[5 3 4 0]
RES_COPY [0 5 0 4]
scan_res_sum [0 8]
SCAN GPU [ 8  5 12]
3
TPB 2 BPG 2
[ 8  5 12  0]
RES_COPY [ 0  8  0 12]
scan_res_sum [ 0 13]
SCAN GPU [10  9 15]
3
TPB 2 BPG 2
[10  9 15  0]
RES_COPY [ 0 10  0 15]
scan_res_sum [ 0 19]
[array([0, 0, 0]), array([0, 5, 8]), array([ 0,  8, 13]), array([ 0, 10, 19])]
[[ 0  0  0  0]
 [ 0  5  8 10]
 [ 0  8 13 19]]
